# Get from API

In this notebook we will get the data needed for the project from the api of [spoonacular](https://spoonacular.com/food-api/docs). We will save this raw data in a csv file, this csv will be cleaned and divided in multiple table and turned in a database in a different notebook.

In [1]:
import pandas as pd
import numpy as np
import requests
import os

In [2]:
# parameters
api_key = 'aaaf78de68d94b1dbd93ee06cc2551fb' # replace with your own API key
params = {
    'number': 100, # limited numbers of returns: 100
    'apiKey': api_key
}

In [3]:
#API call
response = requests.get('https://api.spoonacular.com/recipes/random', params=params) 

for each recipe of the response (if status code = 200), we will take the folowing information:
* id: uniquely identify a recipe
* image: link to an image of the recipe, could be great for the app
* sourceRrl: link to the recipe (for the app)
* title: name of the recipe
* instructions: instruction of the recipe (html format)
* summary: summary of what is the recipe (gluten free?, vegan?, prep time)
* extendedIngredients: ingredients + quantity + id ingrédients + aisle + ...  This part will be cleaned in another notebook

In [4]:
df_list = []  # List to accumulate rows

if response.status_code == 200:
    data = response.json()
    
    for recipe in data['recipes']:
        # Skip the recipe if any info is missing
        if 'id' not in recipe or 'image' not in recipe or 'sourceUrl' not in recipe or 'title' not in recipe or 'instructions' not in recipe or 'summary' not in recipe or 'extendedIngredients' not in recipe:
            continue

        new_row = {
            'id': recipe['id'],
            'image': recipe['image'],
            'sourceUrl': recipe['sourceUrl'],
            'title': recipe['title'],
            'instructions': recipe['instructions'],
            'summary': recipe['summary'],
            'extendedIngredients': recipe['extendedIngredients']
        }
        df_list.append(new_row)

    df = pd.DataFrame(df_list)

    
else:
    print('La requête a échoué avec le code', response.status_code)

In [5]:
#look at the result
print(df.info())
print("duplicates:", df['id'].duplicated().sum())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   100 non-null    int64 
 1   image                100 non-null    object
 2   sourceUrl            100 non-null    object
 3   title                100 non-null    object
 4   instructions         100 non-null    object
 5   summary              100 non-null    object
 6   extendedIngredients  100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB
None
duplicates: 0


,id,image,sourceUrl,title,instructions,summary,extendedIngredients
0,641726,https://spoonacular.com/recipeImages/641726-55...,http://www.foodista.com/recipe/CCFXB5VM/dulce-...,Dulce De Leche Brownies,<ol><li>Preheat the oven to 350 degrees.</li><...,Forget going out to eat or ordering takeout ev...,"[{'id': 18372, 'aisle': 'Baking', 'image': 'wh..."
1,637591,https://spoonacular.com/recipeImages/637591-55...,http://www.foodista.com/recipe/V6PZJBZD/cheese...,Cheese Tortellini Alfredo,<ol><li>In a large pot of salted water cook th...,Cheese Tortellini Alfredo might be a good reci...,"[{'id': 10093727, 'aisle': 'Pasta and Rice;Ref..."
2,662276,https://spoonacular.com/recipeImages/662276-55...,http://www.foodista.com/recipe/V4LG5KBB/sundri...,Sun Dried Tomato and Herb Baked Eggs,<ol><li>Preheat oven to 350 F.</li><li>Cover t...,"If you want to add more <b>gluten free, dairy ...","[{'id': 1123, 'aisle': 'Milk, Eggs, Other Dair..."
3,645315,https://spoonacular.com/recipeImages/645315-55...,http://www.foodista.com/recipe/NCS56WWG/greek-...,Greek Inspired Spring Omelet,<ol><li>In a medium skillet saute asparagus un...,You can never have too many Mediterranean reci...,"[{'id': 11011, 'aisle': 'Produce', 'image': 'a..."
4,660944,https://spoonacular.com/recipeImages/660944-55...,https://www.foodista.com/recipe/JQPV5YW8/spice...,"Spiced Chicken With Risotto, Wild Mushroom Cog...",I prep my vegetables first starting by peelin...,If you want to add more <b>gluten free</b> rec...,"[{'id': 1005091, 'aisle': 'Meat', 'image': 'ch..."
...,...,...,...,...,...,...,...
95,632071,https://spoonacular.com/recipeImages/632071-55...,http://www.foodista.com/recipe/DNFR7JZT/sloppy...,All American Sloppy Joes,<ol><li>Heat the olive oil in a large skillet....,The recipe All American Sloppy Joes could sati...,"[{'id': 6150, 'aisle': 'Condiments', 'image': ..."
96,638035,https://spoonacular.com/recipeImages/638035-55...,http://www.foodista.com/recipe/KFDCB2XH/chicke...,Chicken Cordon Bleu,<ol><li>Preheat oven to 350 degrees F.</li><li...,Chicken Cordon Bleu might be a good recipe to ...,"[{'id': 10010151, 'aisle': 'Meat', 'image': 'h..."
97,648474,https://spoonacular.com/recipeImages/648474-55...,https://www.foodista.com/recipe/D26LR47Y/japan...,Japanese Fried Rice,"In a pan over medium heat, add oil, garlic and...",Japanese Fried Rice is a <b>gluten free and da...,"[{'id': 11304, 'aisle': 'Produce', 'image': 'p..."
98,662376,https://spoonacular.com/recipeImages/662376-55...,http://www.foodista.com/recipe/DMQTR5ZC/superb...,Superbowl Chili,<ol><li>In a large heavy pot set over high hea...,If you want to add more <b>gluten free</b> rec...,"[{'id': 16018, 'aisle': 'Canned and Jarred', '..."


In [6]:
# File path
file_path = '../Data/raw_recipe.csv'

if os.path.isfile(file_path):
    # Append new result to the existing CSV file
    df.to_csv(file_path, mode='a', header=False, index=False)
else:
    # Create a new CSV file and save the result
    df.to_csv(file_path, index=False)


The previous codes have to be run until the number of line minus the number of duplicates satisfy you:

In [7]:
df2 = pd.read_csv('../Data/raw_recipe.csv')
df2.info()
df2['id'].duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1393 non-null   int64 
 1   image                1393 non-null   object
 2   sourceUrl            1393 non-null   object
 3   title                1393 non-null   object
 4   instructions         1369 non-null   object
 5   summary              1393 non-null   object
 6   extendedIngredients  1393 non-null   object
dtypes: int64(1), object(6)
memory usage: 76.3+ KB


365